# Interest Rate Term Structure

這篇文章將介紹在 QuantLib Python 中如何基於給定的存款利率和固定利率債券的收益率，進行收益率曲線的自助法(bootstrapping)建立。所謂自助法，就是從一組給定的利率數據出發，逐步建立完整的收益率曲線。

In [1]:
# 導入 QuantLib 庫
import QuantLib as ql

# 定義列印曲線的函數
def print_curve(xlist, ylist, precision=3):
    print("----------------------")
    print("Maturities\tCurve")
    print("----------------------")
    for x, y in zip(xlist, ylist):
        print(x, "\t\t", round(y, precision))
    print("----------------------")

# 給定存款利率和債券利率
depo_maturities = [ql.Period(6,ql.Months), ql.Period(12, ql.Months)]
depo_rates = [5.25, 5.5]

bond_maturities = [ql.Period(6*i, ql.Months) for i in range(3,21)]
bond_rates = [5.75, 6.0, 6.25, 6.5, 6.75, 6.80, 7.00, 7.1, 7.15,
              7.2, 7.3, 7.35, 7.4, 7.5, 7.6, 7.6, 7.7, 7.8]


首先，我們導入 QuantLib 庫，然後定義一個函數來列印出利率曲線。接下來，我們給定了存款的期限和利率，以及固定利率債券的期限和利率。

In [2]:
# 定義一些常數和約定
calc_date = ql.Date(15, 1, 2015)
ql.Settings.instance().evaluationDate = calc_date

calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
bussiness_convention = ql.Unadjusted
day_count = ql.Thirty360(ql.Thirty360.ISDA)
end_of_month = True
settlement_days = 0
face_amount = 100
coupon_frequency = ql.Period(ql.Semiannual)
settlement_days = 0


定義了一些計算過程中需要的常數和約定，包括計算的日期、日曆、商業約定、日計數方法、是否月底、結算日數、面值以及優惠券的頻率。

In [3]:
# 使用存款利率建立利率助手
depo_helpers = [ql.DepositRateHelper(ql.QuoteHandle(ql.SimpleQuote(r/100.0)),
                                     m,
                                     settlement_days,
                                     calendar,
                                     bussiness_convention,
                                     end_of_month,
                                     day_count )
                for r, m in zip(depo_rates, depo_maturities)]


使用存款利率來建立利率助手。利率助手是用於建立收益率曲線的基本構建塊。

In [4]:
# 使用固定利率債券利率建立固定利率債券助手
bond_helpers = []
for r, m in zip(bond_rates, bond_maturities):
    termination_date = calc_date + m
    schedule = ql.Schedule(calc_date,
                   termination_date,
                   coupon_frequency,
                   calendar,
                   bussiness_convention,
                   bussiness_convention,
                   ql.DateGeneration.Backward,
                   end_of_month)

    helper = ql.FixedRateBondHelper(ql.QuoteHandle(ql.SimpleQuote(face_amount)),
                                        settlement_days,
                                        face_amount,
                                        schedule,
                                        [r/100.0],
                                        day_count,
                                        bussiness_convention,
                                        )
    bond_helpers.append(helper)


使用固定利率債券的利率來建立固定利率債券助手。

In [5]:
# 用兩種助手來建立收益率曲線
rate_helpers = depo_helpers + bond_helpers
yieldcurve = ql.PiecewiseLogCubicDiscount(calc_date,
                             rate_helpers,
                             day_count)


將兩種助手組合在一起來構造收益率曲線。

In [6]:
# 獲取即期利率
spots = []
tenors = []
for d in yieldcurve.dates():
    yrs = day_count.yearFraction(calc_date, d)
    compounding = ql.Compounded
    freq = ql.Semiannual
    zero_rate = yieldcurve.zeroRate(yrs, compounding, freq)
    tenors.append(yrs)
    eq_rate = zero_rate.equivalentRate(day_count,
                                       compounding,
                                       freq,
                                       calc_date,
                                       d).rate()
    spots.append(100*eq_rate)


最後，我們使用 zeroRate 方法從 yieldcurve 物件中獲取即期利率。以上就是利用 QuantLib Python 進行收益率曲線的自助法(bootstrapping)建立的流程。其中，"deposit rate helpers" 和 "fixed rate bond helpers" 是基於給定的利率數據建立的基本構建塊，然後再將這些構建塊組合在一起構造收益率曲線。最後，即期利率可以從收益率曲線中獲取。

In [10]:
print_curve(tenors, spots)

----------------------
Maturities	Curve
----------------------
0.0 		 0.0
0.5 		 5.25
1.0 		 5.426
1.5 		 5.761
2.0 		 6.02
2.5 		 6.283
3.0 		 6.55
3.5 		 6.822
4.0 		 6.87
4.5 		 7.095
5.0 		 7.205
5.5 		 7.257
6.0 		 7.31
6.5 		 7.429
7.0 		 7.485
7.5 		 7.543
8.0 		 7.671
8.5 		 7.802
9.0 		 7.791
9.5 		 7.929
10.0 		 8.072
----------------------


## 核心概念

- "Term Structure" 或稱為 "利率期限結構"，它對於訂價證券是關鍵性的。在 QuantLib 中，我們需要建立一個 YieldTermStructure 物件來進行利率的計算和模擬。
- "Bootstrap" 或稱為 "自助法"，是一種在給定一部分市場數據（如債券價格，存款利率等）的前提下，構造出完整的利率期限結構的方法。
- "Yield Curve" 或稱為 "收益率曲線"，是一種描述不同期限的債券的收益率與其期限之間關係的曲線。在此教學中，我們使用存款利率和債券利率來構造收益率曲線。
- "Spot Rates" 或稱為 "即期利率"，是指無風險投資在特定時間點的利率。即期利率可以用來評價和計算衍生品。